<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/TMLC_LLM_projects/AI_agents/LlamaIndex_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing libraries

In [1]:
!pip install llama-index duckduckgo-search llama-index-llms-openai openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.6/250.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00


## Import libraires, llm and define custom tools

In [2]:
import os
from google.colab import userdata
OpenAI_API = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OpenAI_API

In [3]:
from llama_index.llms.openai import OpenAI  # Import the OpenAI class to interact with OpenAI's language models
from llama_index.core.llms import ChatMessage  # ChatMessage class to structure and handle messages for chat-based interactions
from llama_index.core.tools import BaseTool, FunctionTool  # BaseTool and FunctionTool for building and managing tools within LlamaIndex

In [4]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and return the result."""
    return a * b  # Perform multiplication and return the result

# Create a FunctionTool instance for the 'multiply' function, enabling it to be used as a tool
multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and return the result."""
    return a + b  # Perform addition and return the result

# Create a FunctionTool instance for the 'add' function, enabling it to be used as a tool
add_tool = FunctionTool.from_defaults(fn=add)

In [6]:
llm = OpenAI(model="gpt-4o-mini")  # Initialize an OpenAI language model instance with the specified model ("gpt-4o-mini")

LlamaIndex handles memory by itself

## Function Calling Agent

In [8]:
from llama_index.core.agent import FunctionCallingAgentWorker  # Import a worker class for managing tool-based function calls
from llama_index.core.agent import AgentRunner  # Import the AgentRunner class for running and managing agents

# Initialize a FunctionCallingAgentWorker with the tools and LLM
agent_worker = FunctionCallingAgentWorker.from_tools(
    [multiply_tool, add_tool],  # List of tools (functions wrapped as tools) available to the agent
    llm=llm,  # Specify the language model (e.g., OpenAI's GPT-4o-mini) for the agent
    verbose=True,  # Enable detailed logging for debugging or monitoring agent activities
    allow_parallel_tool_calls=False,  # Restrict the agent to call one tool at a time, preventing parallel executions
)

# Create an AgentRunner to execute tasks using the agent worker
agent = AgentRunner(agent_worker)

In [9]:
response = agent.chat("What is (121 + 2) * 5?")
print(str(response))

Added user message to memory: What is (121 + 2) * 5?
=== Calling Function ===
Calling function: add with args: {"a": 121, "b": 2}
=== Function Output ===
123
=== Calling Function ===
Calling function: multiply with args: {"a": 123, "b": 5}
=== Function Output ===
615
=== LLM Response ===
The result of (121 + 2) * 5 is 615.
The result of (121 + 2) * 5 is 615.


In [10]:
print(response.sources) # output of the tools

[ToolOutput(content='123', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 2}}, raw_output=123, is_error=False), ToolOutput(content='615', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 123, 'b': 5}}, raw_output=615, is_error=False)]


## React Agent

In [13]:
from llama_index.core.agent import ReActAgent  # Import the ReActAgent class, designed for reasoning and acting with tools

# Initialize a ReActAgent with the tools and LLM
agent = ReActAgent.from_tools(
    [multiply_tool, add_tool],  # Provide the list of tools (e.g., multiply and add functions wrapped as tools)
    llm=llm,  # Specify the language model to use for reasoning and generating responses
    verbose=True,  # Enable detailed logging for debugging or understanding the agent's reasoning process
)

# Use the agent to process a chat query involving arithmetic calculations
response = agent.chat("What is (121 * 3) + (5 * 8)?")

# Print the agent's response as a string
print(str(response))  # Convert the response to a string for display

> Running step aa544d40-04d4-479c-a7a9-6cb4d6997dc3. Step input: What is (121 * 3) + (5 * 8)?
Thought: The current language of the user is: English. I need to use the multiply and add tools to help me answer the question.
Action: multiply
Action Input: {'a': 121, 'b': 3}
Observation: 363
> Running step 6de2961b-4e63-40a9-a7e3-6fea18cc268e. Step input: None
Thought: (Implicit) I can answer without any more tools!
Answer: Action: multiply
Action Input: {"a": 5, "b": 8}
Action: multiply
Action Input: {"a": 5, "b": 8}


## Function Calling Agent vs ReAct Agent

The decision between ReACT agents and function-calling agents depends on the nature of the task. For tasks requiring complex reasoning and actions that cannot be easily represented as functions, ReACT agents are often more suitable. On the other hand, if the task involves interacting with specific APIs or performing clearly defined actions, function-calling agents may be a better fit. Both approaches provide powerful tools to enhance LLM capabilities and enable innovative human-machine collaboration.

## Structured Outputs

Special function for OpenAI llms

In [14]:
from llama_index.core.prompts import PromptTemplate  # Import PromptTemplate for creating structured prompts
from pydantic import BaseModel  # Import BaseModel from Pydantic for data modeling
from typing import List  # Import List for type hinting lists of items

# Define a Pydantic model for ticket pricing details
class TicketInfo(BaseModel):
    """Information about ticket pricing."""
    type: str  # Type of ticket (e.g., adult, child)
    price: float  # Price of the ticket

# Define a Pydantic model for opening hours of tourist attractions
class OpeningHours(BaseModel):
    """Opening hours for each day of the week."""
    day: str  # Day of the week (e.g., Monday)
    open_time: str  # Opening time (e.g., "9:00 AM")
    close_time: str  # Closing time (e.g., "5:00 PM")

# Define a Pydantic model for a tourist attraction
class TouristAttraction(BaseModel):
    """A tourist attraction in a city."""
    name: str  # Name of the tourist attraction (e.g., Eiffel Tower)
    city: str  # City where the attraction is located (e.g., Paris)
    description: str  # Description of the attraction
    ticket_info: List[TicketInfo]  # List of TicketInfo objects related to the attraction
    opening_hours: List[OpeningHours]  # List of OpeningHours objects for the attraction

# Initialize the OpenAI LLM with the GPT-3.5 Turbo model
llm = OpenAI(model="gpt-3.5-turbo")

# Define the prompt template to generate details for a tourist attraction in a specified city
prompt_tmpl = PromptTemplate(
    "Generate details of a famous tourist attraction in {city_name}."  # Template that will be formatted with city name
)

# Use the LLM to complete the prompt for a specific city (Paris) and structure the result as a TouristAttraction object
response = (
    llm.as_structured_llm(TouristAttraction)  # Use the LLM with structured output for TouristAttraction
    .complete(prompt_tmpl.format(city_name="Paris"))  # Format the prompt with the city name and generate the response
    .raw  # Get the raw response as a structured object
)

# Print the structured object containing the tourist attraction details
print(response)

name='Eiffel Tower' city='Paris' description='The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is one of the most iconic landmarks in the world and a symbol of France.' ticket_info=[TicketInfo(type='Standard Ticket', price=25.0), TicketInfo(type='Top Floor Ticket', price=50.0)] opening_hours=[OpeningHours(day='Monday', open_time='9:30 AM', close_time='11:45 PM'), OpeningHours(day='Tuesday', open_time='9:30 AM', close_time='11:45 PM'), OpeningHours(day='Wednesday', open_time='9:30 AM', close_time='11:45 PM'), OpeningHours(day='Thursday', open_time='9:30 AM', close_time='11:45 PM'), OpeningHours(day='Friday', open_time='9:30 AM', close_time='11:45 PM'), OpeningHours(day='Saturday', open_time='9:30 AM', close_time='11:45 PM'), OpeningHours(day='Sunday', open_time='9:30 AM', close_time='11:45 PM')]


In [ ]:
display(response)

TouristAttraction(name='Eiffel Tower', city='Paris', description='The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is one of the most iconic landmarks in the world and a symbol of France.', ticket_info=[TicketInfo(type='Standard', price=25.0), TicketInfo(type='Top Floor', price=50.0)], opening_hours=[OpeningHours(day='Monday', open_time='9:30 AM', close_time='11:45 PM'), OpeningHours(day='Tuesday', open_time='9:30 AM', close_time='11:45 PM'), OpeningHours(day='Wednesday', open_time='9:30 AM', close_time='11:45 PM'), OpeningHours(day='Thursday', open_time='9:30 AM', close_time='11:45 PM'), OpeningHours(day='Friday', open_time='9:30 AM', close_time='11:45 PM'), OpeningHours(day='Saturday', open_time='9:30 AM', close_time='11:45 PM'), OpeningHours(day='Sunday', open_time='9:30 AM', close_time='11:45 PM')])

## Agent with Web Search

The code is implemented in 3 different ways:
1. Direct LLM call Agent
2. Function calling Agent
3. Agent Runner Agent

In [16]:
from duckduckgo_search import DDGS

In [15]:
def search(query: str) -> str:

    req = DDGS()  # Initialize the DDGS (DuckDuckGo Search) object to perform the search
    response = req.text(query, max_results=4)  # Perform the search and limit results to 4

    context = ""  # Initialize an empty string to store the combined result texts
    for result in response:  # Iterate over the search results
        context += result['body']  # Concatenate the body of each result into the context

    return context  # Return the concatenated text from the search results

# Create a FunctionTool instance for the search function, making it available as a tool
search_tool = FunctionTool.from_defaults(fn=search)

In [17]:
from llama_index.core import Settings  # Import the Settings class from the llama_index.core module
llm = OpenAI(model="gpt-4o-mini")
Settings.llm = llm # Set the LLM (Language Model) to be used globally in the LlamaIndex framework

### Without tool

In [18]:
print(llm.complete("Who won Border-Gavaskar Trophy (BGT) 2024-2025?").text)

I'm sorry, but I don't have information on events that occurred after October 2023, including the Border-Gavaskar Trophy for 2024-2025. You may want to check the latest sports news or official cricket websites for the most current information.


### Direct LLM call with tool

In [19]:
query = "Who won Border-Gavaskar Trophy (BGT) 2024-2025?"  # Define the query string to ask about the winner of the BGT 2024-2025

# Use the LLM to predict and call the search_tool to find relevant information about the query
function_call = llm.predict_and_call(
    [search_tool],  # List of tools (e.g., search_tool) the LLM can use to gather information
    user_msg=query,  # The user's query message
    allow_parallel_tool_calls=True,  # Allow parallel execution of tool calls (multiple tools can be called at once)
)

# Print the response from the LLM after it has processed the query and used the search tool
print(function_call.response)  # Display the result (likely the winner of the BGT 2024-2025)

Australia won the Border Gavaskar Trophy back after clinching India vs Australia five-match Test series 3-1 at Sydney Cricket Ground on January 5, 2025. A target of 162 could have been trickier had new Test captain Jasprit Bumrah been in a position to bowl despite painful back spasms but once Virat ...Discover the complete list of Border-Gavaskar Trophy winners till 2024. Get insights into the epic India vs Australia Test series, trophy history, and memorable wins.The Border-Gavaskar Trophy 2024-2025 has already delivered thrilling cricketing action, with all five Tests now completed. Played across Australia's iconic venues, this five-Test series challenged players with its demanding format, testing their skill, strategy, and endurance.The Border-Gavaskar Trophy (colloquially known as BGT) [3] is an International Test cricket trophy played between India and Australia. The series is named after distinguished former captains, Australia's Allan Border and India's Sunil Gavaskar. It is pla

### Function Calling

In [ ]:
from llama_index.core.agent import FunctionCallingAgent, FunctionCallingAgentWorker  # Import agent classes for managing tool calls

# Initialize the FunctionCallingAgent with the specified tools and LLM
agent = FunctionCallingAgent.from_tools(
    [search_tool],  # Provide the list of tools (e.g., search_tool) that the agent can use
    llm=llm,  # Specify the LLM (e.g., OpenAI model) to be used for reasoning and generating responses
    verbose=True,  # Enable verbose mode to log detailed agent actions and reasoning steps
    allow_parallel_tool_calls=True,  # Allow the agent to call multiple tools in parallel if needed
)

# Define the query to be processed by the agent
query = "Who won Border-Gavaskar Trophy (BGT) 2024-2025?"

# Use the agent to process the query and generate a response
response = agent.chat(query)  # The agent processes the query, utilizes tools, and returns the response

# Print the response generated by the agent
print(response)  # Display the result, which should contain the answer to the query (e.g., BGT winner)

> Running step 05d452e4-98a0-4104-98f9-1d88218a710e. Step input: Who won Border-Gavaskar Trophy (BGT) 2024-2025?
Added user message to memory: Who won Border-Gavaskar Trophy (BGT) 2024-2025?
=== Calling Function ===
Calling function: search with args: {"query": "Border-Gavaskar Trophy 2024-2025 winner"}
=== Function Output ===
Australia won the Border Gavaskar Trophy back after clinching India vs Australia five-match Test series 3-1 at Sydney Cricket Ground on January 5, 2025. A target of 162 could have been trickier had new Test captain Jasprit Bumrah been in a position to bowl despite painful back spasms but once Virat ...The winner of a Test series wins the trophy. If a series is drawn, the country holding the trophy retains it. ... As of January 2025, Australia hold the trophy after they defeated India 3-1 in the 2024-25 series. ... 2024-25 series The Border-Gavaskar Trophy is a prestigious Test cricket series contested between India and Australia, named ...Discover the complete li

### Agent Runner over Function Calling Agent

In [ ]:
from llama_index.core.agent import AgentRunner  # Import the AgentRunner class to manage agent workflows

# Initialize the FunctionCallingAgentWorker with the specified tools and LLM
agent_worker = FunctionCallingAgentWorker.from_tools(
    [search_tool],  # Provide the list of tools (e.g., search_tool) that the agent can use
    llm=llm,  # Specify the LLM (e.g., OpenAI model) to be used for reasoning and generating responses
    verbose=True,  # Enable verbose mode to log detailed agent actions and reasoning steps
    allow_parallel_tool_calls=True,  # Allow the agent to call multiple tools in parallel if needed
)

# Create an AgentRunner instance using the agent worker
agent_runner = AgentRunner(agent_worker)  # AgentRunner will manage the execution of the agent workflow

# Define the query to be processed by the agent
query = "Who won Border-Gavaskar Trophy (BGT) 2024-2025?"

# Use the AgentRunner to process the query and generate a response
runner_chat = agent_runner.chat(query)  # The agent runner invokes the agent and handles the response

# Print the response generated by the agent
print(runner_chat.response)  # Display the result, which should contain the answer to the query (e.g., BGT winner)

Added user message to memory: Who won Border-Gavaskar Trophy (BGT) 2024-2025?
=== Calling Function ===
Calling function: search with args: {"query": "Border-Gavaskar Trophy 2024-2025 winner"}
=== Function Output ===
The winner of a Test series wins the trophy. If a series is drawn, the country holding the trophy retains it. ... As of January 2025, Australia hold the trophy after they defeated India 3-1 in the 2024-25 series. ... 2024-25 series The Border-Gavaskar Trophy is a prestigious Test cricket series contested between India and Australia, named ...Australia won the Border Gavaskar Trophy back after clinching India vs Australia five-match Test series 3-1 at Sydney Cricket Ground on January 5, 2025. A target of 162 could have been trickier had new Test captain Jasprit Bumrah been in a position to bowl despite painful back spasms but once Virat ...Discover the complete list of Border-Gavaskar Trophy winners till 2024. Get insights into the epic India vs Australia Test series, trophy